In [22]:
from datasets import load_from_disk
dataset = load_from_disk('./model3-outputs-gpt2tokenizer-askubuntubody-margintesting')
#dataset = load_from_disk('./model3-outputs-gpt2tokenizer-askubuntubody-margintraining-150000train-10000test')
from mymodelmargin import MySimilarityModel
#mymodel = MySimilarityModel.from_pretrained('./nodotmodel3_weights_2024-08-13--01:38:03alaki')
mymodel = MySimilarityModel.from_pretrained('./nodotmodel3_weights_2024-08-13--19:33:49alaki')

Im here!
Im after that!


In [23]:
mydict = dataset['validation'][:5]
collator_input = [dict(zip(mydict.keys(), values)) for values in zip(*[mydict[col] for col in mydict.keys()])]
collator_input[0].keys()
    

dict_keys(['similar', 'random', 'input_ids'])

In [24]:
from mycollators import DataCollatorForTextSimilarity
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
mycollator = DataCollatorForTextSimilarity(tokenizer, 10)
outputs = mycollator(collator_input)
mymodel.to('cuda')
list_of_sub_batches = [sub_batch.to('cuda') for sub_batch in outputs['list_of_sub_batches']]
model_outputs = mymodel(list_of_sub_batches, outputs['sign'], outputs['indices'])

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [28]:
print('similar scores are ', model_outputs['similar_scores'][4])
print('random scores are ', model_outputs['random_scores'][4])

similar scores are  tensor([0.3667, 0.4063], device='cuda:0', grad_fn=<SumBackward1>)
random scores are  tensor([0.3663, 0.3945, 0.3486, 0.3631, 0.3146, 0.3482, 0.3340, 0.4246, 0.4210,
        0.3299, 0.4211, 0.3544, 0.2430, 0.3667, 0.2777, 0.4063, 0.3877, 0.3866,
        0.3870, 0.3712], device='cuda:0', grad_fn=<SumBackward1>)


In [5]:
from openaimodel import OpenAIModel
openAImodel = OpenAIModel()
model_outputs = openAImodel(list_of_sub_batches, outputs['sign'], outputs['indices'])


im here i is  0  j is  0
im here i is  0  j is  1
im here i is  0  j is  2
im here i is  0  j is  3
im here i is  0  j is  4
im here i is  0  j is  5
im here i is  0  j is  6
im here i is  0  j is  7
im here i is  0  j is  8
im here i is  0  j is  9
im here i is  1  j is  0
im here i is  1  j is  1
im here i is  1  j is  2
im here i is  1  j is  3
im here i is  1  j is  4
im here i is  1  j is  5
im here i is  1  j is  6
im here i is  1  j is  7
im here i is  1  j is  8
im here i is  1  j is  9
im here i is  2  j is  0
im here i is  2  j is  1
<built-in method count of str object at 0x7d119bfa3b90>


In [24]:
## lading AskUbuntu Question Dataset!
import pandas as pd
df = pd.read_csv('texts_raw_fixed.txt', sep='\t', header=None)
df.columns = ['qid', 'title', 'body']
similar_train_df = pd.read_csv('train_random.txt', sep='\t', header=None)
similar_train_df.columns = ['qid', 'similar', 'random']
similar_test_df = pd.read_csv('test.txt', sep='\t', header=None)
similar_test_df.columns = ['qid', 'similar', 'random', 'bm25-scores']